和TCP较大的不同点在于：

1. 它是一种无连接 协议

也就是说：无需事先建立虚拟连接，可以直接给对方地址发送消息。

通信方的地址也是由 IP地址 和 端口号构成。

所以相比 TCP协议，它更加简单快捷。

2. 没有消息可靠性保证

UDP传输的消息如果在网络上丢失了，就丢失了。UDP协议本身没有重传机制。

而TCP协议底层有消息验证是否到达，如果丢失，发送方会重传的机制。

所以，如果应用要么不在意丢失一些信息，要么应用层自己实现一套机制保证可靠性。

3. 数据消息发送是独立的报文

TCP协议通信双方的信息数据就像流动在管道中，是有明确的先后次序的。

发送方应用 先发送的信息肯定 是 先被接收方应用 先接收的 。

而UDP协议发送的是一个个的独立的报文，接收方应用接收到的次序不一定和发送的次序一致。


* TCP Socket 是流式（stream）协议，如果应用接收缓冲不够大，只接受了一部分，没有关系，后面继续接收，然后找到消息边界拼接就可以了。

* 而 UDP式数据报协议， UDP Socket 如果只接受了数据报的一部分，剩余的消息就会被丢弃。下次接收，只能接收到下一个数据报的内容了。* 



### 服务端代码

In [ ]:
import socket,json

BUFF_LEN     = 400                   # 最大报文长度
SERVER_ADDR  = ("127.0.0.1", 18000)  # 指明服务端地址

# 创建 UDP Socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# 设置socket超时时间，单位：秒
client_socket.settimeout(2)

# 要发送的信息 对象
message = {
    'action' : '获取信息',
    'name' : '白月黑羽'
} 
# 发送出去的信息必须是字节，所以要先序列化，再编码
sendbytes = json.dumps(message).encode('utf8')
client_socket.sendto(sendbytes, SERVER_ADDR)
try:
    recvbytes, server = client_socket.recvfrom(BUFF_LEN)
    # 接收到的信息是字节，所以要解码，再反序列化
    message = json.loads(recvbytes.decode('utf8'))
    print(message)
except socket.timeout:
    print('接收消息超时')

### 服务端代码

import socket,json

BUFF_LEN = 400    # 最大报文长度
ADDR     = ("", 18000)  # 指明服务端地址，IP地址为空表示本机所有IP

# 创建 UDP Socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
# 绑定地址
server_socket.bind(ADDR)

while True:
    try:
        recvbytes, client_addr = server_socket.recvfrom(BUFF_LEN)
    except socket.timeout:
        continue
    
    print(f'来自 {client_addr} 的请求')

    # 接收到的信息是字节，所以要解码，再反序列化
    message = json.loads(recvbytes.decode('utf8'))
    print(message)
    if message['action'] == '获取信息':
        # 可以从数据库的数据源查询 此用户的信息
        username = message['name']

        # 要发送的信息 对象
        message = {
            'action' : '返回信息',
            'info' : f'{username} 的信息是:xxxxxxxx'
        } 
        # 发送出去的信息必须是字节，所以要先序列化，再编码
        sendbytes = json.dumps(message).encode('utf8')
        server_socket.sendto(sendbytes, client_addr)


可以看出 UDP通信服务端 的socket 也需要绑定端口号。

但是和TCP不同， 服务端只需要一个socket进行通信即可，不需要 2个socket分别用来监听和通信。


而 UDP客户端的socket 通常不需要指定绑定的端口号， 操作系统会自动帮其选择一个绑定。


当不需要使用 UDP Socket 时，可以通过 socket 对象的 close 方法 关闭，如下代码所示。

server_socket.close()
关闭 socket 后，该端口绑定的 端口号 就会被释放，可以再次被本进程或者其它进程的 socket 绑定使用